In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("../dataset/emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1) 

'\n# Build the CNN model\nmodel = Sequential()\nmodel.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length))\nmodel.add(Conv1D(128, 5, activation=\'relu\'))\nmodel.add(MaxPooling1D(5))\nmodel.add(Conv1D(128, 5, activation=\'relu\'))\nmodel.add(GlobalMaxPooling1D())\nmodel.add(Dense(num_classes, activation=\'softmax\'))\n\n# Compile the model\nmodel.compile(loss=\'categorical_crossentropy\', optimizer=\'adam\', metrics=[\'accuracy\'])\n\n# Train the model\nearly_stopping = EarlyStopping(monitor=\'val_loss\', patience=3, restore_best_weights=True)\nmodel.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])\n\n# Evaluate the model\nloss, accuracy = model.evaluate(X_test, y_test)\nprint("Test Loss:", loss)\nprint("Test Accuracy:", accuracy)\n\n# Predict labels\ny_pred = model.predict(X_test)\ny_pred = np.argmax(y_pred, axis=1)\ny_true = np.argmax(np.array(y_test), axis=1)\n\n# Calculate precision, recall, a

In [1]:
from keras.models import load_model

model = load_model('emotions_cnn_model (1).keras')

2024-05-01 13:09:14.690419: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 13:09:14.723832: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 13:09:14.967538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 13:09:14.967716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 13:09:15.000511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

2606/2606 [==============================] - 25s 9ms/step - loss: 0.3303 - accuracy: 0.8530
Test Loss: 0.3302783668041229
Test Accuracy: 0.8530385494232178


In [5]:
# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

2606/2606 [==============================] - 25s 9ms/step
Precision: 0.8601776840371637
Recall: 0.7915775954590413
F1-score: 0.8189751282996055
